# Ejemplo(Falta agregar un costo de almacenamiento)
Juan vende cervezas artesanales de Lunes a Viernes. Las compra por 300 pesos y vende por 1.500 pesos cada una
Juan maneja su inventario de forma semanal, de la siguiente manera:
* Juan parte la semana con una cantidad de cervezas en inventario, y vende lascervezas durante la semana.
    
* Si un cliente quiere comprar una cerveza pero no hay disponible, Juan consideraque pierde 300 pesos por cada cerveza no vendida por falta de clientes.
    
* Juan vende entre 10 y 40 cervezas por semana.
    
* El Viernes al final del día Juan hace la contabilidad de costos, y cuenta cuantas unidades tiene en inventario. Según su nivel de inventario, tiene que tomar una decisión.
    
    * Compra lo suficiente para llenar su frigobar, que tiene una capacidad de S unidades.
    * No hacer ningún pedido esta semana.
* El pedido llega el Lunes en la mañana.
* S=50
* Juan pide dinero prestado, por lo cual considera una tasa de descuento del dinero de un 10% semanal

## ¿Ayude a decidir a Juan cuando comprar, según el inventario que tenga?

In [2]:
import numpy as np
from numpy import random
import collections

In [3]:
# Información problema
S=5#00#20#0 #capacidad max y tamaño de lote
a=0#00#0#3#0 #Demanda minima
b=5#00#15#0 #Demanda maxima
cost=300#precio cerveza y costo perdida clientes
price=1500
taza=0.9


#### Generó las demandas

In [4]:
random.seed(10)
total=50000
x=random.randint(a,b+1, size=(total))
Dem_frec = collections.Counter(x)
Dem_frec

Counter({1: 8382, 5: 8209, 4: 8338, 0: 8215, 3: 8499, 2: 8357})

### Comiezo a generar mi matriz de probaibilidades
Esta matriz se estructura de la siguiente forma

    Prob[i][j]: es la probailidad de que el inventario i pase al nivel de inventario j en el siguente periodo


In [5]:
prob =np.zeros([S+1,S+1])

In [6]:
prob#[5][5]=1

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [7]:
for i in range(S+1):
    for j in range(S+1):
        dem=i-j
        if(S>=dem>=0):
            if(dem<i):
               prob[i][j] =Dem_frec[dem]/total
    prob[i][0] = 1 - sum(prob[i][0:])


In [8]:
prob

array([[1.     , 0.     , 0.     , 0.     , 0.     , 0.     ],
       [0.8357 , 0.1643 , 0.     , 0.     , 0.     , 0.     ],
       [0.66806, 0.16764, 0.1643 , 0.     , 0.     , 0.     ],
       [0.50092, 0.16714, 0.16764, 0.1643 , 0.     , 0.     ],
       [0.33094, 0.16998, 0.16714, 0.16764, 0.1643 , 0.     ],
       [0.16418, 0.16676, 0.16998, 0.16714, 0.16764, 0.1643 ]])

### primera idea
Esta matriz de probabilidades no sirve para calcular el rebenue esperado, porque no representa la demanda cuando sobrepasa el nivel de stock

Para almacenar el expected reward, se utilizará al siguiente estructura:

    reward[i][d]:= el expected reward de tomar la decisión d ante un nivel de inventario i.
    
tomando en cuenta que d, puede tener valores de 0 o 1., sea 0 no realizar la compra y 1 realizar la compra de inventario.

## Esta idea es mejor
tomando en cuenta que realizar la compra en el periodo t, donde el nivel de inventarios es de i, no afectaria al expected reward del mismo periodo t, ya que la compra afecta al nivel de inventarios del perido siguente. Es por esto que la desición no afecta al expected reward del mismo periodo en donde se toma.

Es por ello que para almacenar el expected reward, se utilizará al siguiente estructura:
    
    reward[i]:= el expected reward ante un nivel de inventario i.


In [9]:
# aquí se ajustan los costos de
# D>i o D<i

reward=np.zeros(S+1)
for i in range(S+1):
    r=0
    for dem in range(a,b+1):
        p=Dem_frec[dem]/total
        r_dem=price*min(dem,i)
        #if(dem>=i):
            #r_dem+=-cost*(dem-i)
        #else:
        if(dem<i):#Agrego costo de inventario sobrante
            r_dem+= -(cost)*(i-dem)
        r+= p*(r_dem)
    #print(r)
    reward[i]=r

# Comenzamos a estructurar esto

## Value Determination Equations

In [10]:
#funcion para obtener el valor de la ecuaciones
def Value_Determination_Equations(decisions,prob,descuento,costo,revenues):
    L=len(decisions)
    aux=np.zeros([L,L])
    B=np.zeros(L)
    for i in range(L):
        if decisions[i]:
        #Juan decide comprar mas cerveza.
            aux[i]=prob[L-1]
            B[i]=revenues[L-1]#antes era i
            if(S-i>0):
            # si el nivel de inventario es menor al S el costo de pedir puede ser
                B[i]+=-costo*(S-i)
        else:
            aux[i]=prob[i]
            B[i]=revenues[i]
    A = np.identity(L)-(descuento*aux)
    X = np.linalg.solve(A,B)
    return X
    

In [11]:
D=np.zeros(6)
#D=np.array([0,0,1,1,1,1])

Value_Determination_Equations(D,prob,taza,cost,reward)

array([-1.05856046e-12,  1.41323507e+03,  2.72257901e+03,  3.90994699e+03,
        4.95203708e+03,  5.82487506e+03])

# Howard’s Policy Iteration Method

In [55]:
def T_delta(prob,values,descuento,costo,revenues):
    L=len(values)
    T=np.zeros(L)
    #aux_1 siempre igual
   
    aux_1=revenues[L-1]+(descuento*sum(prob[L-1]*values))#compro
    for i in range(L):
        aux_2=revenues[i]+(descuento*sum(prob[i]*values))
        T[i]=max(aux_1-costo*(L-1-i),aux_2)#Con i=S soy indiferente a comprar o no
    return T

In [ ]:
""""
def T_delta(prob,values,descuento,costo,revenues):
    L=len(values)
    T=np.zeros(L)
    #aux_1 siempre igual
    aux_1=costo+(descuento*sum(prob[0]*values))
    for i in range(L):
        aux_2=revenues[i]+(descuento*sum(prob[i]*values))
        T[i]=max(aux_1,aux_2)
    return T

In [51]:
D=np.array([1., 1., 1., 1., 0., 1.])

In [52]:
V_D_E=Value_Determination_Equations(D,prob,taza,cost,reward)

In [53]:
Los_T=T_delta(prob,V_D_E,taza,cost,reward)
np.around(Los_T,4)==np.around(V_D_E,4)

array([ True,  True,  True,  True,  True,  True])

In [35]:
print(V_D_E)
print(Los_T)

[21865.48089437 22165.48089437 22465.48089437 22765.48089437
 23144.04931769 23365.48089437]
[21865.48089437 22165.48089437 22465.48089437 22765.48089437
 23144.04931769 23365.48089437]


In [14]:
def Search_OP(decisions,prob,descuento,costo,revenues):
   
    #(D,m_prob,descuento,c,states_reveneus)
    V_D_E = Value_Determination_Equations(decisions,prob,descuento,costo,revenues)
    Los_T=T_delta(prob,V_D_E,descuento,costo,revenues)
    val_Op=np.around(Los_T,4)==np.around(V_D_E,4)
    D_p = np.copy(decisions)
    if(sum(val_Op)==len(val_Op)):
        return(decisions)
    else:
        for i in range(len(val_Op)):
            if (val_Op[i]!=True):
                if(decisions[i]==1):
                    D_p[i]=0
                else:
                    D_p[i]=1
                    #V_D_Prima = Value_Determination_Equations(D_p,prob,descuento,costo,revenues)
                decisions[i]=D_p[i]
                return(Search_OP(decisions,prob,descuento,costo,revenues))
        #For 
        #print('recurción')        
        

In [54]:
#D=np.array([1,0,0,0,0,1])
D=np.ones(S+1)
#D=np.array([1 for i in range(S+1)])
Optimo=Search_OP(D,prob,taza,cost,reward)
Optimo

array([1., 1., 1., 1., 0., 1.])

In [16]:
sum(Optimo)

4.0

In [17]:
#D=np.array([1,0,0,0,0,1])
#D=np.zeros(501)
D=np.array([1 for i in range(S+1)])
Optimo=Search_OP(D,prob,taza,cost,reward)
print(sum(Optimo))
Optimo

5


array([1, 1, 1, 1, 0, 1])